In [1]:
import glob
import librosa
import os
import sys
from pathlib import Path
from glob import glob
from os.path import join, exists, basename, splitext

## Encoder model数据处理

ST-CMD数据处理

In [ ]:
cd /search/hadoop07/wuzijun/voice_data/STcmds

In [ ]:
file_list = glob.glob('./ST-CMDS-20170001_1-OS/*.wav')

In [ ]:
splitext(basename(file_list[0]))

In [ ]:
speaker_list = set([splitext(basename(f))[0][9:15] for f in file_list])

In [ ]:
len(speaker_list)

In [ ]:
for speaker in speaker_list:
    Path(f'./wav/{speaker}').mkdir(exist_ok=True)
    os.system(f'mv ./ST-CMDS-20170001_1-OS/*{speaker}* ./wav/{speaker}/')

Prime数据处理

In [ ]:
cd ../Primewords/

In [ ]:
import pandas as pd
import json

In [ ]:
js = json.load(open('/search/hadoop07/wuzijun/voice_data/Primewords/set1_transcript.json', 'r'))

In [ ]:
wave_data = pd.DataFrame(js)

In [ ]:
wave_data.head(10)

In [ ]:
max(wave_data.user_id.apply(lambda x: len(str(x))))

In [ ]:
wave_data.dtypes

In [ ]:
wave_data.user_id = wave_data.user_id.apply(lambda x: str(10000 + int(x))[-4:])

In [ ]:
len(set(wave_data.user_id))

In [ ]:
wave_data.to_csv('./transcript/transcript.txt', header=None, index=None)

In [ ]:
find ./audio_files/ -type f -exec mv {}  ./wav/ \;

In [ ]:
for speaker in set(wave_data.user_id):
    Path(f'./wav/{speaker}').mkdir(exist_ok=True)

In [ ]:
wave_data[['file', 'user_id']] \
    .apply(lambda x: os.system(f'mv ./wav/{x[0]} ./wav/{x[1]}'), axis=1)

## Synthesizer model数据处理

In [ ]:
from pypinyin import pinyin, lazy_pinyin, Style

In [ ]:
pinyin('算了吧', style=Style.TONE3)

In [ ]:
from pypinyin.contrib.neutral_tone import NeutralToneWith5Mixin
from pypinyin.converter import DefaultConverter
from pypinyin.core import Pinyin

In [ ]:
class MyConverter(NeutralToneWith5Mixin, DefaultConverter):
    pass

In [ ]:
my_pinyin = Pinyin(MyConverter())
pinyin = my_pinyin.pinyin
lazy_pinyin = my_pinyin.lazy_pinyin

In [ ]:
lazy_pinyin('我不喜欢吃薄荷', style=Style.TONE3)

### AIshell

In [ ]:
!head -3 /search/hadoop07/wuzijun/voice_data/Biaobei/audio_text_val_filelist.txt

In [ ]:
path_aishell = '/search/hadoop07/wuzijun/voice_data/AIshell'

In [ ]:
ll $path_aishell/wav/S0002

In [ ]:
!head -3 $path_aishell/transcript/aishell_transcript_v0.8.txt

In [ ]:
speaker_aishell = glob(f'{path_aishell}wav/*')

In [ ]:
len(speaker_aishell)

In [ ]:
transcripts_aishell = open(f'{path_aishell}/transcript/aishell_transcript_v0.8.txt', 'r').readlines()

In [ ]:
def get_pinyin(sentence):
    sentence = sentence.strip().replace(' ', '').replace('\n', '')
    return ' '.join(lazy_pinyin(sentence, style=Style.TONE3))

In [ ]:
def get_path(root_path, first):
    speaker_path = first[6:11]
    file_name = first + '.wav'
    return join(root_path, speaker_path, file_name)

In [ ]:
aishell_tuples = [(get_path(join(path_aishell, 'wav'), l[:16]), get_pinyin(l[16:]), l[6:11]) for l in transcripts_aishell]

In [ ]:
len(aishell_tuples)

In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [ ]:
aishell_train, aishell_eval = train_test_split(aishell_tuples, test_size=300)

In [ ]:
len(aishell_train), len(aishell_eval)

In [ ]:
aishell_train_df = pd.read_csv(f'{path_aishell}/audio_text_train_filelist.txt', 
                               sep='|', header=None, 
                               names=['wav_path', 'pinyin', 'speaker_name', 'speaker_emb_path'])

In [ ]:
aishell_train_df.speaker_emb_path = aishell_train_df.speaker_emb_path.apply(lambda x: x.replace('AIshell/', 'AIshell/embed/'))

In [ ]:
aishell_val_df = pd.read_csv(f'{path_aishell}/audio_text_val_filelist.txt', 
                               sep='|', header=None, 
                               names=['wav_path', 'pinyin', 'speaker_name', 'speaker_emb_path'])

In [ ]:
aishell_val_df.speaker_emb_path = aishell_val_df.speaker_emb_path.apply(lambda x: x.replace('AIshell/', 'AIshell/embed/'))

In [ ]:
aishell_train_df = pd.DataFrame(aishell_train, columns=['wav_path', 'pinyin', 'SpeakerID'])

In [ ]:
aishell_eval_df = pd.DataFrame(aishell_eval, columns=['wav_path', 'pinyin', 'SpeakerID'])

In [ ]:
aishell_eval_df.head(1)

In [ ]:
aishell_train_df['speaker_emb_path'] = aishell_train_df[['wav_path', 'SpeakerID']] \
    .apply(lambda x: x[0].replace('wav', 'embed').replace('/' + x[1], ''), axis=1)

In [ ]:
aishell_eval_df['speaker_emb_path'] = aishell_eval_df[['wav_path', 'SpeakerID']] \
    .apply(lambda x: x[0].replace('wav', 'embed').replace('/' + x[1], ''), axis=1)

In [ ]:
aishell_train_df.to_csv(f'{path_aishell}/audio_text_train_filelist.txt', sep='|', header=False, index=False)

In [ ]:
aishell_val_df.to_csv(f'{path_aishell}/audio_text_val_filelist.txt', sep='|', header=False, index=False)

In [87]:
aishell_trans = pd.read_csv('/search/hadoop07/wuzijun/voice_data/AIshell/transcript/aishell_transcript_v0.8.txt', sep='     ', names=['file_name', 'text'])

/search/hadoop02/suanfa/anaconda3/envs/tacotron2/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [88]:
aishell_trans.head(3)

,file_name,text
0,BAC009S0002W0122 而 对 楼市 成交 抑制 作用 最 大 的 限 购,None
1,BAC009S0002W0123 也 成为 地方 政府 的 眼中 钉,None
2,BAC009S0002W0124 自 六月 底 呼和浩特 市 率先 宣布 取消 限 购 后,None


### Magicdata

In [ ]:
import pandas as pd

In [ ]:
path_magic = '/search/hadoop07/wuzijun/voice_data/Magicdata'

In [ ]:
cat $path_magic/transcript/*.txt >> $path_magic/transcript/all_trans.txt

In [ ]:
magic_list = pd.read_csv(path_magic + '/transcript/all_trans.txt', sep='\t')

In [ ]:
magic_list = magic_list[magic_list.UtteranceID.apply(lambda x: '.wav' in x)]

In [ ]:
magic_list.drop_duplicates(subset='UtteranceID', inplace=True)

In [ ]:
len(magic_list)

In [ ]:
magic_list.head(3)

In [ ]:
path_magic

In [ ]:
magic_list['wav_path'] = magic_list[['UtteranceID', 'SpeakerID']].apply(lambda x: f'{path_magic}/wav/{x[1]}/{x[0]}', axis=1)

In [ ]:
magic_list['pinyin'] = magic_list.Transcription.apply(lambda x: get_pinyin(x))

In [ ]:
magic_list['speaker_emb_path'] = magic_list[['wav_path', 'SpeakerID']] \
    .apply(lambda x: x[0].replace('wav', 'embed').replace(x[1] + '/', ''), axis=1)

In [ ]:
magic_list.speaker_emb_path[0], magic_list.wav_path[0]

In [ ]:
magic_trans = magic_list[['wav_path', 'pinyin', 'SpeakerID', 'speaker_emb_path']].copy()

In [ ]:
magic_train, magic_val = train_test_split(magic_trans, test_size=300)

In [ ]:
len(magic_train), len(magic_val)

In [ ]:
magic_train = pd.read_csv(path_magic + '/audio_text_train_filelist.txt', header=None, sep='|', 
                          names=['wav_path', 'pinyin', 'Speaker_name', 'speaker_emb_path'])

In [ ]:
magic_val = pd.read_csv(path_magic + '/audio_text_val_filelist.txt', header=None, sep='|', 
                          names=['wav_path', 'pinyin', 'Speaker_name', 'speaker_emb_path'])

In [ ]:
magic_train.speaker_emb_path[0]

In [ ]:
magic_train.speaker_emb_path = magic_train.speaker_emb_path.apply(lambda x: x.replace('Magicdata/embed', 'Magicdata/embed/'))

In [ ]:
magic_val.speaker_emb_path = magic_val.speaker_emb_path.apply(lambda x: x.replace('Magicdata/embed', 'Magicdata/embed/'))

In [ ]:
magic_train.to_csv(path_magic + '/audio_text_train_filelist.txt', header=None, sep='|', index=False)

In [ ]:
magic_val.to_csv(path_magic + '/audio_text_val_filelist.txt', header=None, sep='|', index=False)

### Biaobei

In [ ]:
biaobei_train = pd.read_csv('/search/hadoop07/wuzijun/voice_data/Biaobei/audio_text_train_filelist.txt', 
                            sep='|', header=None, names=['wav_path', 'pinyin', 'speaker_name', 'speaker_emb_path'])

In [ ]:
biaobei_val = pd.read_csv('/search/hadoop07/wuzijun/voice_data/Biaobei/audio_text_val_filelist.txt', 
                            sep='|', header=None, names=['wav_path', 'pinyin', 'speaker_name', 'speaker_emb_path'])

In [ ]:
biaobei_train.speaker_emb_path = biaobei_train.speaker_emb_path.apply(lambda x: x.replace('Biaobei/', 'Biaobei/embed/'))

In [ ]:
biaobei_val.speaker_emb_path = biaobei_val.speaker_emb_path.apply(lambda x: x.replace('Biaobei/', 'Biaobei/embed/'))

In [ ]:
biaobei_val.speaker_emb_path[0]

In [ ]:
biaobei_train['speaker_name'] = 'Biaobei'
biaobei_val['speaker_name'] = 'Biaobei'

In [ ]:
biaobei_train['speaker_emb_path'] = biaobei_train.wav_path.apply(lambda x: x.replace('Wave', 'wav').replace('wav', 'embed'))

In [ ]:
biaobei_val['speaker_emb_path'] = biaobei_val.wav_path.apply(lambda x: x.replace('Wave', 'wav').replace('wav', 'embed'))

In [ ]:
biaobei_train.speaker_emb_path[0], biaobei_train.wav_path[0]

In [ ]:
biaobei_train.reset_index(inplace=True)

In [ ]:
biaobei_train.wav_path = biaobei_train.level_0
biaobei_train.pinyin = biaobei_train.level_1

In [ ]:
biaobei_train = biaobei_train[['wav_path', 'pinyin', 'speaker_emb_path']].copy()

In [ ]:
biaobei_train['speaker_name'] = 'Biaobei'

In [ ]:
biaobei_train.wav_path = biaobei_train.wav_path.apply(lambda x: x.replace('Wave', 'wav'))

In [ ]:
biaobei_train[['wav_path', 'pinyin', 'speaker_name', 'speaker_emb_path']].to_csv('/search/hadoop07/wuzijun/voice_data/Biaobei/audio_text_train_filelist.txt', sep='|', index=False, header=False)

In [ ]:
biaobei_val.reset_index(inplace=True)

In [ ]:
biaobei_val.wav_path = biaobei_val.level_0
biaobei_val.pinyin = biaobei_val.level_1

In [ ]:
biaobei_val = biaobei_val[['wav_path', 'pinyin', 'speaker_emb_path']].copy()

In [ ]:
biaobei_val['speaker_name'] = 'Biaobei'

In [ ]:
biaobei_val.wav_path = biaobei_val.wav_path.apply(lambda x: x.replace('Wave', 'wav'))

In [ ]:
biaobei_val[['wav_path', 'pinyin', 'speaker_name', 'speaker_emb_path']].to_csv('/search/hadoop07/wuzijun/voice_data/Biaobei/audio_text_val_filelist.txt', sep='|', index=False, header=False)

## 测试

In [ ]:
test1 = np.load('/search/hadoop07/wuzijun/voice_data/Biaobei/embed/009996.npy')

In [ ]:
test2 = np.load('/search/hadoop07/wuzijun/voice_data/Biaobei/embed/009991.npy')

In [ ]:
import librosa
import IPython
from IPython.display import Audio

In [ ]:
path = '/search/hadoop07/wuzijun/voice_data/Magicdata/wav/15_3621/15_3621_20170810140303.wav'

In [ ]:
IPython.display.display(Audio(librosa.load(path, sr=16000)[0], rate=16000))

In [ ]:
import torch

In [ ]:
from torch import nn

In [ ]:
nn.utils.rnn.pad_packed_sequence

In [ ]:
a = torch.rand(5, 9, 4)

In [ ]:
b = torch.rand(5, 9, 2)

In [ ]:
torch.Tensor.expand

In [ ]:
torch.zeros(1).repeat(3,4)

In [ ]:
from encoder.model import SpeakerEncoder

In [ ]:
model = SpeakerEncoder('cuda', 'cuda')

In [ ]:
import torch

In [ ]:
checkpoint = torch.load('encoder/saved_models/pretrained.pt')

In [ ]:
import numpy as np

In [ ]:
from synthesizer.utils.audio import AudioProcessor

In [ ]:
from synthesizer.utils.generic_utils import load_config

In [ ]:
c = load_config('synthesizer/config.json')

In [ ]:
ap = AudioProcessor(**c.audio)

In [ ]:
ap._stft_parameters()

In [ ]:
import numpy as np

In [ ]:
import librosa

In [ ]:
import soundfile

In [ ]:
x, sr = soundfile.read('/search/hadoop07/wuzijun/voice_data/Biaobei/wav/000001.wav')

In [ ]:
x, len(x)

In [ ]:
x1, sr1 = librosa.load('/search/hadoop07/wuzijun/voice_data/Biaobei/wav/000001.wav', sr=16000)

In [ ]:
x1, sr1

In [ ]:
len(x1)

In [ ]:
x2 = librosa.resample(x, sr, 16000)

In [ ]:
x2

In [ ]:
len(x2)

In [ ]:
x2 == x1

In [ ]:
cd /search/hadoop07/wuzijun/voice_data/AIshell/embed/

In [ ]:
pwd

In [ ]:
import glob

In [ ]:
test = glob.glob('*')

In [ ]:
to_rename = [n for n in test if '\n' in n]

In [ ]:
to_rename[0]

In [ ]:
len(to_rename)

In [ ]:
import os, sys

In [ ]:
for n in to_rename:
    new_n = n.replace('\n', '')
    prefix = new_n.replace('.npy', '').split('/')[-1]
    # print(f'{n} => {new_n}')
    os.system(f"mv {prefix}*.npy {new_n}")

In [ ]:
1200//2

In [ ]:
p = Path('./bt')

In [ ]:
p.

In [ ]:
TypeError

In [ ]:
import numpy as np

In [ ]:
t = np.load('/search/hadoop07/wuzijun/voice_data/Biaobei/WaveGAN/all/000012-feats.npy')

In [ ]:
t.shape

In [85]:
l = [4,5,6]

In [86]:
for i, l in enumerate(l, 1):
    print(i, l)

1 4
2 5
3 6


In [4]:
os.path.join("/search/hadoop07/wuzijun/voice_data/SV2TTS/synthesizer/", "mels/11")

'/search/hadoop07/wuzijun/voice_data/SV2TTS/synthesizer/mels/11'

In [2]:
import numpy as np

In [3]:
np.load('../Data/voice_data/Biaobei/Mels/mel/009100.npy').shape

(80, 521)

In [9]:
a = np.load('../Data/voice_data/SV2TTS/synthesizer/mels/mel-BAC009S0025W0125_00.npy')

In [10]:
a.shape

(251, 80)

In [12]:
a.T.shape

(80, 251)